In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from src.experiment import get_fallacy_identification_df, save_fallacy_identification_df
from src.analysis import score_fallacy_identification


In [ ]:
df = get_fallacy_identification_df()

In [ ]:
score_fallacy_identification(df)

df.head()

In [ ]:
df['gpt_4o_mini_score'].value_counts()

In [ ]:
df['gpt_4_score'].value_counts()

In [ ]:
df.info()

In [ ]:
score_columns = [col for col in df.columns if col.endswith('_score')]

percent_scores = np.round(df[score_columns].mean() * 100, 1)
percent_scores = percent_scores.reset_index()
percent_scores.columns = ['Score Column', 'Percent Score']

percent_scores

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme()

sns.barplot(x='Score Column', y='Percent Score', data=percent_scores)
plt.title('Percent Scores for Each Column')
plt.ylabel('Percent Score (%)')
plt.xlabel('Score Columns')
plt.show()
